In [1]:
from sklearn.metrics import confusion_matrix, silhouette_score, calinski_harabasz_score, homogeneity_score, completeness_score, v_measure_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense, Lambda
from sklearn.ensemble import IsolationForest
from keras.models import Sequential, Model
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.linear_model import LogisticRegression
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import OneClassSVM
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras import backend as K
from pandas import DataFrame
from sklearn.metrics import accuracy_score
import seaborn as sb
import pandas as pd
import numpy as np
import sklearn
import random
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
# loading the dataset
pd.options.display.max_columns = None
data = pd.read_csv('Deloitte ML Challenge Predict Loan Defaulters (Kaggle).csv')
data.head(5)

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,Home Ownership,Verification Status,Payment Plan,Loan Title,Debit to Income,Delinquency - two years,Inquires - six months,Open Account,Public Record,Revolving Balance,Revolving Utilities,Total Accounts,Initial List Status,Total Received Interest,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,65087372,10000,32236,12329.36286,59,BAT2522922,11.135007,B,C4,MORTGAGE,176346.62670,Not Verified,n,Debt Consolidation,16.284758,1,0,13,0,24246,74.932551,7,w,2929.646315,0.102055,2.498291,0.793724,0,INDIVIDUAL,49,0,31,311301,6619,0
1,1450153,3609,11940,12191.99692,59,BAT1586599,12.237563,C,D3,RENT,39833.92100,Source Verified,n,Debt consolidation,15.412409,0,0,12,0,812,78.297186,13,f,772.769385,0.036181,2.377215,0.974821,0,INDIVIDUAL,109,0,53,182610,20885,0
2,1969101,28276,9311,21603.22455,59,BAT2136391,12.545884,F,D4,MORTGAGE,91506.69105,Source Verified,n,Debt Consolidation,28.137619,0,0,14,0,1843,2.073040,20,w,863.324396,18.778660,4.316277,1.020075,0,INDIVIDUAL,66,0,34,89801,26155,0
3,6651430,11170,6954,17877.15585,59,BAT2428731,16.731201,C,C3,MORTGAGE,108286.57590,Source Verified,n,Debt consolidation,18.043730,1,0,7,0,13819,67.467951,12,w,288.173196,0.044131,0.107020,0.749971,0,INDIVIDUAL,39,0,40,9189,60214,0
4,14354669,16890,13226,13539.92667,59,BAT5341619,15.008300,C,D4,MORTGAGE,44234.82545,Source Verified,n,Credit card refinancing,17.209886,1,3,13,1,1544,85.250761,22,w,129.239553,19.306646,1294.818751,0.368953,0,INDIVIDUAL,18,0,430,126029,22579,0


In [3]:
# Checking for missing data
missing_values = data.isnull().sum()
print(missing_values)

ID                              0
Loan Amount                     0
Funded Amount                   0
Funded Amount Investor          0
Term                            0
Batch Enrolled                  0
Interest Rate                   0
Grade                           0
Sub Grade                       0
Employment Duration             0
Home Ownership                  0
Verification Status             0
Payment Plan                    0
Loan Title                      0
Debit to Income                 0
Delinquency - two years         0
Inquires - six months           0
Open Account                    0
Public Record                   0
Revolving Balance               0
Revolving Utilities             0
Total Accounts                  0
Initial List Status             0
Total Received Interest         0
Total Received Late Fee         0
Recoveries                      0
Collection Recovery Fee         0
Collection 12 months Medical    0
Application Type                0
Last week Pay 

In [4]:
data = data.drop(['ID','Accounts Delinquent','Payment Plan','Loan Title'], axis=1)

In [5]:
le = LabelEncoder()
data['Term'] = le.fit_transform(data['Term'])
data['Batch Enrolled'] = le.fit_transform(data['Batch Enrolled'])
data['Grade'] = le.fit_transform(data['Grade'])
data['Sub Grade'] = le.fit_transform(data['Sub Grade'])
data['Employment Duration'] = le.fit_transform(data['Employment Duration'])
data['Verification Status'] = le.fit_transform(data['Verification Status'])
data['Initial List Status'] = le.fit_transform(data['Initial List Status'])
data['Application Type'] = le.fit_transform(data['Application Type'])

In [6]:
data.head(5)

,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,Home Ownership,Verification Status,Debit to Income,Delinquency - two years,Inquires - six months,Open Account,Public Record,Revolving Balance,Revolving Utilities,Total Accounts,Initial List Status,Total Received Interest,Total Received Late Fee,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,10000,32236,12329.36286,2,16,11.135007,1,13,0,176346.62670,0,16.284758,1,0,13,0,24246,74.932551,7,1,2929.646315,0.102055,2.498291,0.793724,0,0,49,31,311301,6619,0
1,3609,11940,12191.99692,2,4,12.237563,2,17,2,39833.92100,1,15.412409,0,0,12,0,812,78.297186,13,0,772.769385,0.036181,2.377215,0.974821,0,0,109,53,182610,20885,0
2,28276,9311,21603.22455,2,11,12.545884,5,18,0,91506.69105,1,28.137619,0,0,14,0,1843,2.073040,20,1,863.324396,18.778660,4.316277,1.020075,0,0,66,34,89801,26155,0
3,11170,6954,17877.15585,2,15,16.731201,2,12,0,108286.57590,1,18.043730,1,0,7,0,13819,67.467951,12,1,288.173196,0.044131,0.107020,0.749971,0,0,39,40,9189,60214,0
4,16890,13226,13539.92667,2,32,15.008300,2,18,0,44234.82545,1,17.209886,1,3,13,1,1544,85.250761,22,1,129.239553,19.306646,1294.818751,0.368953,0,0,18,430,126029,22579,0


In [7]:
# Splitting Data
X = data.drop(['Loan Status'], axis = 1)
y = data['Loan Status']

In [8]:
unique_values, counts = np.unique(y, return_counts=True)

# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

Value: 0, Count: 61222
Value: 1, Count: 6241


In [9]:
X.shape

(67463, 30)

# FEATURE SELECTION using Contrastive Learning

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

class SiameseNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(SiameseNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 28),
            nn.ReLU(),
            nn.Linear(28, 26),
            nn.ReLU(),
            nn.Linear(26, output_size)
        )

    def forward(self, x1, x2):
        out1 = self.fc(x1)
        out2 = self.fc(x2)
        return out1, out2


class ContrastiveLoss(nn.Module):
    def __init__(self, margin):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

siamese_network = SiameseNetwork(input_size=30, output_size=24)
contrastive_loss = ContrastiveLoss(margin=1.0)
optimizer = optim.Adam(siamese_network.parameters(), lr=0.001)

class SiameseDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        x1 = self.X[index]
        label = self.y[index]
        x2 = self.X[torch.randint(len(self.X), (1,)).item()]
        return x1, x2, torch.tensor(label, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

batch_size = 64
train_dataset = SiameseDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

num_epochs = 1
for epoch in range(num_epochs):
    for x1, x2, label in train_loader:
        optimizer.zero_grad()
        output1, output2 = siamese_network(x1, x2)
        loss = contrastive_loss(output1, output2, label)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

siamese_network.eval()
with torch.no_grad():
    num_samples = len(X_tensor)
    indices = torch.randint(num_samples, (num_samples,))
    x1_eval = X_tensor
    x2_eval = X_tensor[indices]

    all_outputs, _ = siamese_network(x1_eval, x2_eval)

    
columns = [f'CL{i+1}' for i in range(24)]
df_CL = pd.DataFrame(data=all_outputs, columns=columns)

Epoch [1/1], Loss: 0.008227250538766384


# SSML

In [11]:
X = df_CL
X_labeled, X_unlabeled, y_labeled, _ = train_test_split(X, y, test_size=0.9, random_state=42)
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_unlabeled)
pseudo_labels = kmeans.fit_predict(X_unlabeled)
X_combined = np.vstack([X_labeled, X_unlabeled])
y_combined = np.concatenate([y_labeled, pseudo_labels])


classifier = LogisticRegression(random_state=42)
classifier.fit(X_combined, y_combined)
y_pred = classifier.predict(X_combined)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y, y_pred))

# Print evaluation metrics
calinski_harabasz_index = calinski_harabasz_score(df_CL, y_pred)
homogeneity = homogeneity_score(y, y_pred)
completeness = completeness_score(y, y_pred)
v_measure = v_measure_score(y, y_pred)


print(f"Calinski-Harabasz Index: {calinski_harabasz_index}")
print(f"Homogeneity: {homogeneity}")
print(f"Completeness: {completeness}")
print(f"V-Measure: {v_measure}")


Confusion Matrix:
[[32332 28890]
 [ 3212  3029]]
Calinski-Harabasz Index: 1.2253875150225484
Homogeneity: 9.873145124906424e-05
Completeness: 4.400672974875008e-05
V-Measure: 6.087857169604808e-05
